# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [1]:
import sys
import json
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import torch
import joblib
from tqdm.autonotebook import tqdm
from sklearn.preprocessing import StandardScaler
from utils.config_utils import load_and_merge_configs_for_notebook

# --- 1. 环境与路径设置 ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    print(f"将项目根目录添加到 sys.path: {project_root}")
    sys.path.append(project_root)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 2.  加载配置和所有模块 ---
# a. 加载配置
config = load_and_merge_configs_for_notebook()

# b. 手动加载所有模块
print("--- 正在加载所有项目模块... ---")
try:
    from main_train import (
        run_all_data_pipeline, run_preprocess_l3_cache, run_hpo_train,
        run_all_models_train, run_performance_evaluation, run_results_visualization,
    )
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df, get_latest_global_data
    from data_process.save_data import run_data_pipeline, get_processed_data_path
    from model.build_models import run_training_for_ticker
    from utils.hpo_utils import run_hpo_for_ticker
    from model.builders.model_fuser import ModelFuser
    from model.builders.lgbm_builder import LGBMBuilder
    from model.builders.lstm_builder import LSTMBuilder, LSTMModel
    from model.builders.tabtransformer_builder import TabTransformerBuilder, TabTransformerModel
    from risk_management.risk_manager import RiskManager
    from utils.date_utils import resolve_data_pipeline_dates
    from utils.encoding_utils import encode_categorical_features
    from utils.file_utils import find_latest_artifact_paths
    from utils.ml_utils import walk_forward_split
    print("INFO: 项目模块导入成功。")
except ImportError as e:
    raise ImportError(f"模块导入失败，请确保 Notebook 的运行目录在项目根目录。错误: {e}")

# c. 构建 modules 字典
modules = {
    'initialize_apis': initialize_apis, 'shutdown_apis': shutdown_apis,
    'get_full_feature_df': get_full_feature_df, 'get_latest_global_data': get_latest_global_data,
    'run_data_pipeline': run_data_pipeline, 'get_processed_data_path': get_processed_data_path,
    'run_training_for_ticker': run_training_for_ticker, 'run_hpo_for_ticker': run_hpo_for_ticker,
    'ModelFuser': ModelFuser, 'LGBMBuilder': LGBMBuilder, 'LSTMBuilder': LSTMBuilder, 'LSTMModel': LSTMModel,
    'TabTransformerBuilder': TabTransformerBuilder, 'TabTransformerModel': TabTransformerModel,
    'RiskManager': RiskManager,
    'resolve_data_pipeline_dates': resolve_data_pipeline_dates,
    'encode_categorical_features': encode_categorical_features,
    'find_latest_artifact_paths': find_latest_artifact_paths,
    'walk_forward_split': walk_forward_split,
    'pd': pd, 'torch': torch, 'joblib': joblib, 'tqdm': tqdm, 'StandardScaler': StandardScaler, 'Path': Path, 'yaml': yaml, 'json': json
}


C:\Users\Akari\AppData\Local\Temp\ipykernel_32720\2077795851.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


将项目根目录添加到 sys.path: D:\Project\Command\Python\Neural\Wolf_of_Wall_Street
--- 正在为 Notebook 加载和合并所有配置文件 ---
  - 主配置 'config.yaml' 已加载。
  - 子配置 'data\default.yaml' 已加载到 'data' 组。
  - 子配置 'features\default.yaml' 已加载到 'features' 组。
  - 子配置 'labeling\raw_return.yaml' 已加载到 'labeling' 组。
  - 子配置 'model\lgbm.yaml' 已加载到 'model' 组。
  - 子配置 'hpo\default.yaml' 已加载到 'hpo' 组。
  - 子配置 'backtest\default.yaml' 已加载到 'backtest' 组。
  - 子配置 'application\default.yaml' 已加载到 'application' 组。
--- 所有配置文件合并成功 ---
--- 正在加载所有项目模块... ---
INFO: 项目模块导入成功。


# **阶段一：数据准备与特征工程**

In [2]:
USE_LATEST_DATE = True
if config and modules:
    print(f"--- INFO: 日期模式 -> {'使用最新日期' if USE_LATEST_DATE else '使用配置文件中的固定日期'} ---")
    run_all_data_pipeline(config, modules, use_today_as_end_date=USE_LATEST_DATE)

--- INFO: 日期模式 -> 使用最新日期 ---
=== 阶段一：数据准备与特征工程 ===
INFO: 已启用动态日期模式，将使用今天的日期 '2025-10-28' 作为 end_date。
INFO: 日期已解析. Start Date: 2010-10-28, End Date: 2025-10-28
INFO: 正在尝试登录 Baostock...
login success!
INFO: Baostock API 登录成功。
INFO: 未在配置中提供有效的 Tushare Token，将跳过 Tushare 相关数据。
开始执行数据管道协调任务...
WARNNING: 股票池为空，无需处理。
logout success!
INFO: Baostock API 已成功登出.
--- 阶段 1 成功完成。 ---


# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [3]:
if config and modules:
    # force_reprocess=True/False 可以控制是否重建缓存
    run_preprocess_l3_cache(config, modules, force_reprocess=False)

=== 工作流阶段 2.1：为模型预处理数据 (L3 缓存) ===
INFO: 开始执行预处理流程 (分块保存模式)...

--- INFO: L3 缓存文件已在磁盘上准备就绪。 ---
--- 阶段 2.1 成功完成。 ---


### 2.2 超参数优化

In [4]:
RUN_HPO = False
if RUN_HPO and config and modules:
    run_hpo_train(config, modules)

### 2.3 模型训练

In [5]:
if config and modules:
    all_ic_history = run_all_models_train(
        config, 
        modules,  
        force_retrain_base=False, 
        force_retrain_fuser=True
    )

=== 工作流阶段 2.3：训练所有模型 ===
ERROR: 配置为空或股票池为空，无法训练模型。


### 2.4 结果聚合、评估与可视化

In [6]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)
    else: print('无可视化结果')